In [1]:
!pip list

Package                Version
---------------------- -------------------
absl-py                0.10.0
aiohttp                3.6.3
argon2-cffi            20.1.0
astor                  0.8.1
async-generator        1.10
async-timeout          3.0.1
attrs                  20.2.0
backcall               0.2.0
bayesian-optimization  1.2.0
bleach                 3.3.0
blinker                1.4
boto3                  1.17.46
botocore               1.20.78
brotlipy               0.7.0
cachetools             4.1.1
certifi                2020.12.5
cffi                   1.14.3
chardet                3.0.4
click                  7.1.2
cloudpickle            1.6.0
colorama               0.4.4
cryptography           3.1.1
cycler                 0.10.0
decorator              5.0.9
deepcut                0.7.0.0
defusedxml             0.7.1
entrypoints            0.3
et-xmlfile             1.1.0
Flask                  1.1.2
gast                   0.2.2
gensim                 3.6.0
google-auth      

In [1]:
!pip install numpy==1.19.5 --user

In [2]:
!pip install h5py==2.10.0 --user

In [3]:
!pip install gensim==3.6.0 --user

In [4]:
!pip install deepcut --user

In [5]:
!pip install pythainlp --user

In [6]:
!pip install nltk --user

In [12]:
#prediction
import deepcut
import pandas as pd
import numpy as np
import re
from itertools import chain
import itertools
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Lambda
import tensorflow.keras.backend as K

In [13]:
# Question import
import requests

# Category model import
import pickle
import scipy.sparse as sp
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import TruncatedSVD
import joblib

In [14]:
# from flask_ngrok import run_with_ngrok
from flask import Flask
import threading 
import json
import time

## Category_model

In [15]:
data = pd.read_excel("Category.xlsx")
data

,Category
0,หลักสูตร
1,คำถามทั่วไป
2,ลงทะเบียนเรียน
3,การรับเข้านักศึกษา


In [16]:
#Load File
with open('token_text_category.data', 'rb') as filehandle:
    # read the data as binary data stream
    tokenized_texts = pickle.load(filehandle)

In [17]:
def tokenize_text_list(ls):
    """Tokenize list of text"""
    return list(chain.from_iterable([deepcut.tokenize(ls)]))

In [18]:
def text_to_bow(tokenized_text, vocabulary_):
    n_doc = len(tokenized_text)
    values, row_indices, col_indices = [], [], []
    for r, tokens in enumerate(tokenized_text):
        feature = {}
        for token in tokens:
            word_index = vocabulary_.get(token)
            if word_index is not None:
                if word_index not in feature.keys():
                    feature[word_index] = 1
                else:
                    feature[word_index] += 1
        for c, v in feature.items():
            values.append(v)
            row_indices.append(r)
            col_indices.append(c)
        #print(feature)

    # document-term matrix in sparse CSR format
    X = sp.csr_matrix((values, (row_indices, col_indices)),
                      shape=(n_doc, len(vocabulary_)))
    return X

vocabulary_ = {v: k for k, v in enumerate(set(chain.from_iterable(tokenized_texts)))}
X = text_to_bow(tokenized_texts, vocabulary_)

In [19]:
transformer = TfidfTransformer()
svd_model = TruncatedSVD(n_components=100,
                         algorithm='arpack', n_iter=100)
X_tfidf = transformer.fit_transform(X)
X_svd = svd_model.fit_transform(X_tfidf)

In [20]:
tag = pd.get_dummies(data.Category).columns

In [21]:
#Load Model
logist_models = joblib.load("category_model.pkl")

C:\Users\NATTHAWATTUNGRUETHAI\.conda\envs\tensorflow\lib\site-packages\sklearn\base.py:315: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [22]:
y_pred = np.argmax(np.vstack([model.predict_proba(X_svd)[:, 1] for model in logist_models]).T, axis=1)
y_pred = np.array([tag[yi] for yi in y_pred])
y_true = data.Category.values
print(tag[0:4])

Index(['การรับเข้านักศึกษา', 'คำถามทั่วไป', 'ลงทะเบียนเรียน', 'หลักสูตร'], dtype='object')


## Prediction

In [23]:
#Clean Text
def remove_repettition(text):
    token_list = list(text)
    if len(token_list) > 2:
        filter_list = [True, True]
        n = len(token_list)
        for i in range(2, n):
            if (token_list[i] == token_list[i-1]) and (token_list[i] == token_list[i-2]):
                filter_list.append(False)
            else:
                filter_list.append(True)

        output = ''.join(np.array(token_list)[filter_list])
    else:
        output = text
    return output

def cleansing(text):
    # \t, \n, \xa0 and other special characters. Replace by blank string
    text = re.sub('[\t\n\xa0\"\'!?\/\(\)%\:\=\-\+\*\_ๆ]', '', text)
    
    # Numbers. Replace by space
    text = re.sub('[0-9]', ' ', text)
    
    # Dot. Replace by space
    text = re.sub('[\.]', ' ', text)
    
    # One or more consecutive space. Replace by single space
    text = re.sub('\s+',' ',text)
    
    # Remove 2 or more repettition
    text = remove_repettition(text)
    
    return text

In [24]:
import gensim
wv_model = gensim.models.Word2Vec.load('corpus.th.model')

In [25]:
def word2idx(word):
    index = 0
    index = wv_model.wv.vocab[word].index
    return index

In [26]:
def word_index(listword):
    dataset = []
    vocabulary = dict()
    inverse_vocabulary = ['<unk>']  # '<unk>' will never be used, it is only a placeholder for the [0, 0, ....0] embedding
    for sentence in listword:
        tmp = []
        for w in sentence:
            if w not in wv_model:
                continue

            if w not in vocabulary:
                vocabulary[w] = len(inverse_vocabulary)
                tmp.append(len(inverse_vocabulary))
                inverse_vocabulary.append(w)
            else:
                tmp.append(word2idx(w))
        dataset.append(tmp)
    return np.array(dataset)

In [27]:
# define word embedding
vocab_list = [(k, wv_model.wv[k]) for k, v in wv_model.wv.vocab.items()]
embeddings_matrix = np.zeros((len(wv_model.wv.vocab.items()) + 1, wv_model.vector_size))
for i in range(len(vocab_list)):
    word = vocab_list[i][0]
    embeddings_matrix[i + 1] = vocab_list[i][1]

In [28]:
# vocab_list

In [29]:
EMBEDDING_DIM = 300
embeddings_matrix = 1 * np.random.randn(len(vocab_list) + 1, EMBEDDING_DIM)  # This will be the embedding matrix
embeddings_matrix[0] = 0  # So that the padding will be ignored

In [30]:
# Model variables
n_hidden = 256
batch_size = 128
n_epoch = 100
max_seq_length = 2704

In [31]:
# embeddings_matrix

In [32]:
def exponent_neg_manhattan_distance(left, right):
    ''' Helper function for the similarity estimate of the LSTMs outputs'''
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

In [33]:
# The visible layer
left_input = Input(shape=(max_seq_length,), dtype='int32')
right_input = Input(shape=(max_seq_length,), dtype='int32')

embedding_layer = Embedding(len(embeddings_matrix), EMBEDDING_DIM, weights=[embeddings_matrix], input_length=max_seq_length, trainable=False)

# Embedded version of the inputs
encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

# Since this is a siamese network, both sides share the same LSTM
shared_lstm = LSTM(n_hidden)

left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)

# Calculates the distance as defined by the MaLSTM model
malstm_distance = Lambda(function=lambda x: exponent_neg_manhattan_distance(x[0], x[1]),output_shape=lambda x: (x[0][0], 1))([left_output, right_output])

# Pack it all up into a model
malstm = Model([left_input, right_input], [malstm_distance])


malstm.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

# Start training
training_start_time = time.time()

In [34]:
malstm.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 2704)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 2704)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 2704, 300)    9468300     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 256)          570368      embedding[0][0]              

In [35]:
# Load best weight from model
malstm.load_weights('sm_colab_ka.h5')

#Test with Text

In [36]:
def prepare_for_predict(input_questions):
    q_input= []
    cleansing(input_questions)
    tokenized_input_1 =deepcut.tokenize(input_questions)
    for sentence in tokenized_input_1:
      q_input.append(sentence)
    q_input= word_index(tokenized_input_1)
    q_input = pad_sequences(q_input, maxlen=max_seq_length)
    return q_input

In [37]:
max_word = 19219
max_seq_length = 2704

In [38]:
#Duplicate list
def duplicate(testList, n):
    return [ele for ele in testList for _ in range(n)]

## data from all category

In [39]:
exit_flag = False
beforeTok={}
tokenized = {}

def getQuestions():
    global beforeTok
    global tokenized
    while True:
#         raw_questions = backendAPI()
        raw_questions = getQuestionsFromBackendAPI() # <-- Uncomment this to get real questions
        
        # do tokenize
        tokenize_questions = raw_questions
        
        curriculumDF = pd.DataFrame(data=tokenize_questions['หลักสูตร'])
        curriculumDF = curriculumDF.rename(columns={0:"curriculum"})
        
        admissionDF = pd.DataFrame(data=tokenize_questions['การรับเข้านักศึกษา'])
        admissionDF = admissionDF.rename(columns={0:"admission"})
        
        enrollmentDF = pd.DataFrame(data=tokenize_questions['ลงทะเบียนเรียน'])
        enrollmentDF = enrollmentDF.rename(columns={0:"enrollment"})
        
        faqDF = pd.DataFrame(data=tokenize_questions['คำถามทั่วไป'])
        faqDF = faqDF.rename(columns={0:"faq"})
        
        tokenized_enrollment =enrollmentDF.enrollment.map(tokenize_text_list)
        tokenized_admission =admissionDF.admission.map(tokenize_text_list)
        tokenized_curriculum =curriculumDF.curriculum.map(tokenize_text_list)
        tokenized_faq =faqDF.faq.map(tokenize_text_list)
        
        tokenized = {}
        tokenized['ลงทะเบียนเรียน'] = tokenized_enrollment
        tokenized['การรับเข้านักศึกษา'] = tokenized_admission
        tokenized['หลักสูตร'] = tokenized_curriculum
        tokenized['คำถามทั่วไป'] = tokenized_faq
        
#         beforeTok = {}
        beforeTok['ลงทะเบียนเรียน'] = enrollmentDF
        beforeTok['การรับเข้านักศึกษา'] = admissionDF
        beforeTok['หลักสูตร'] = curriculumDF
        beforeTok['คำถามทั่วไป'] = faqDF
        
        # Update question
        questions_data = tokenize_questions
        if exit_flag: 
            break
        
        # Set query time
        time.sleep(259200)

In [40]:
url = 'https://natthawat.live/api'

In [41]:
def getQuestionsFromBackendAPI():
    response = requests.get('%s/km/faq' % url)
    faqs = json.loads(response.text)

    response = requests.get('%s/km/category' % url)
    categories = json.loads(response.text)

    questions_data = {}
    for category in categories:
        questions_data[category['category']] = []
        for faq in faqs:
            if faq['category']['category'] == category['category']:
                questions_data[category['category']].append(faq['question'])
                
    return questions_data

In [42]:
getQuestionsThread = threading.Thread(target = getQuestions)

In [43]:
getQuestionsThread.start()

In [46]:
print(beforeTok)

{'ลงทะเบียนเรียน':                                            enrollment
0   สามารถลงทะเบียนเกินหน่วยกิตที่หลักสูตรกำหนดได้...
1    ใบแจ้งชำระเงินค่าลงทะเบียน สั่งพิมพ์ได้จากที่ไหน
2                        ตอนดรอปต้องขอลายเซ็นใครบ้าง?
3                       ขอทรานสคริปต์ออนไลน์ได้ที่ไหน
4                     ยังต้องเข้าไปเรียนที่มหาลัยไหม?
5       ลงทะเบียนเรียนเพิ่มรายวิชา ชำระเงินได้ที่ไหน?
6   ชำระเงินค่าลงทะเบียนแล้ว แต่ในระบบยังขึ้นว่าไม...
7   ถ้าจะดึงบางวิชาลงมาเรียนแทนตัวที่ติด f ได้ไหม ...
8   ลืมจ่ายเงินค่าลงทะเบียน และ ใบแจ้งหนี้เกินกำหน...
9   ถ้าต้องการลงทะเบียนเรียนน้อยกว่า 9 หน่วยกิตต้อ...
10                   ถ้าการย้ายหลักสูตรต้องทำอย่างไร?
11                            ตรวจวันเวลาสอบได้ที่ไหน
12  ขอลดรายวิชาออนไลน์ หลังหมดเขต เพิ่ม-ลด 2 สัปดา...
13  กรณีเพิ่มและหลักสูตรปิดรายวิชา จะได้รับเงินคืน...
14                      แจ้งขอสำเร็จการศึกษาได้ที่ไหน
15  ถ้าต้องการลงทะเบียนเรียนมากกว่า 19 หน่วยกิตต้อ...
16                               วิชาเลือกไหนน่าสนใจ?
17       

# time total

In [47]:
%%time
inputQuestion = "วิศวคอมมีหลักสูตรอะไรบ้าง"
# print('input: ' + inputQuestion)

#category model
inputQuestion= cleansing(inputQuestion)
tokenized_input_2 = inputQuestion
tokenized_text = deepcut.tokenize(tokenized_input_2)
x = text_to_bow([tokenized_text], vocabulary_)
x_tfidf = transformer.transform(x)
x_svd = svd_model.transform(x_tfidf)
pred = [model.predict_proba(x_svd.reshape(-1, 1).T).ravel()[1] for model in logist_models]

# print(list(zip(tag, pred)))
predict_category = max(list(zip(tag, pred)))
max_value = 0
max_category = ''
pred_results = list(zip(tag, pred))

for pred_result in pred_results:
    # print(pred_result)
    if pred_result[1] > max_value:
        max_value = pred_result[1]
        max_category = pred_result[0]
# print(max_category, max_value)
# end of category model


# print(tokenized[max_category])


# prediction
tokenized_category = tokenized[max_category]
# print(tokenized_category)
max_word = 19219
max_seq_length = 2704
q_category= []
for sentence in tokenized_category:
    q_category.append(sentence)
q_category = word_index(q_category)
all_Question_categorylen = len(q_category)
# all_Question_categorylen

tokenized_dup_input_2= duplicate([tokenized_input_2],all_Question_categorylen)
# print(tokenized_dup_input_2)

q_user = word_index(tokenized_dup_input_2)
# Split to dicts
M_input = {'left': q_category, 'right': q_user}
# Zero padding
for model_input, side in itertools.product([M_input], ['left', 'right']):
    model_input[side] = pad_sequences(model_input[side], maxlen=max_seq_length)

# Make sure everything is ok
assert M_input['left'].shape == M_input['right'].shape
play_predict = malstm.predict(x=[M_input['left'], M_input['right']])

max_question_percentage = max(play_predict)
# print(max_question_percentage)
question_index = np.where(play_predict == max_question_percentage)
# print(question_index)
predictedQuestion = beforeTok[max_category].loc[question_index[0][0]]
predictedQuestion = predictedQuestion[0]

value = {
    "category": max_category,
    "accuracy": "%lf" % max_value,
    "predictedQuestion": str(predictedQuestion),
    "similarity": "%lf" % max_question_percentage
}

C:\Users\NATTHAWATTUNGRUETHAI\.conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  
C:\Users\NATTHAWATTUNGRUETHAI\.conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Wall time: 1.68 s


# time seperate

In [47]:
%%time
inputQuestion = "วิศวคอมมีหลักสูตรอะไรบ้าง"
print('input: ' + inputQuestion)

input: วิศวคอมมีหลักสูตรอะไรบ้าง
Wall time: 0 ns


In [48]:
%%time
#category model
tokenized_input_2= cleansing(inputQuestion)

Wall time: 0 ns


In [49]:
%%time
tokenized_text = deepcut.tokenize(tokenized_input_2)

Wall time: 46.8 ms


In [50]:
%%time
x = text_to_bow([tokenized_text], vocabulary_)
x_tfidf = transformer.transform(x)
x_svd = svd_model.transform(x_tfidf)

Wall time: 1.99 ms


In [51]:
%%time
pred = [model.predict_proba(x_svd.reshape(-1, 1).T).ravel()[1] for model in logist_models]
# print(list(zip(tag, pred)))

Wall time: 0 ns


In [52]:
%%time
predict_category = max(list(zip(tag, pred)))
max_value = 0
max_category = ''
pred_results = list(zip(tag, pred))

Wall time: 0 ns


In [53]:
%%time
for pred_result in pred_results:
    # print(pred_result)
    if pred_result[1] > max_value:
        max_value = pred_result[1]
        max_category = pred_result[0]
print(max_category, max_value)
# end of category model

หลักสูตร 0.430342087763526
Wall time: 0 ns


In [54]:
%%time
# prediction
tokenized_category = tokenized[max_category]
# print(tokenized_category)
max_word = 19219
max_seq_length = 2704

Wall time: 0 ns


In [55]:
%%time
q_category= []
for sentence in tokenized_category:
    q_category.append(sentence)
q_category = word_index(q_category)
all_Question_categorylen = len(q_category)
# all_Question_categorylen

Wall time: 4.01 ms


C:\Users\NATTHAWATTUNGRUETHAI\.conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  
C:\Users\NATTHAWATTUNGRUETHAI\.conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [56]:
%%time
tokenized_input_2 = deepcut.tokenize(inputQuestion)

Wall time: 47.9 ms


In [57]:
%%time
tokenized_dup_input_2= duplicate([tokenized_input_2],all_Question_categorylen)

Wall time: 0 ns


In [58]:
%%time
q_user = word_index(tokenized_dup_input_2)
# Split to dicts
M_input = {'left': q_category, 'right': q_user}
# Zero padding
for model_input, side in itertools.product([M_input], ['left', 'right']):
    model_input[side] = pad_sequences(model_input[side], maxlen=max_seq_length)

Wall time: 3.99 ms


C:\Users\NATTHAWATTUNGRUETHAI\.conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  


In [59]:
%%time
# Make sure everything is ok
assert M_input['left'].shape == M_input['right'].shape

Wall time: 0 ns


In [60]:
%%time
play_predict = malstm.predict(x=[M_input['left'],  M_input['right']])

Wall time: 707 ms


In [61]:
%%time
max_question_percentage = max(play_predict)
print(max_question_percentage)
question_index = np.where(play_predict == max_question_percentage)
print(question_index)
predictedQuestion = beforeTok[max_category].loc[question_index[0][0]]
predictedQuestion = predictedQuestion[0]

[0.973298]
(array([42], dtype=int64), array([0], dtype=int64))
Wall time: 2 ms


In [62]:
%%time
value = {
    "category": max_category,
    "accuracy": "%lf" % max_value,
    "predictedQuestion": str(predictedQuestion),
    "similarity": "%lf" % max_question_percentage
}

Wall time: 0 ns
